In [2]:
import seaborn as sns
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
%pylab inline
import os
from __future__ import print_function
from __future__ import division
import os, sys
# hook up all scripts
sys.path.append(os.path.abspath('../scripts')) # if notebook in folder
sys.path.append(os.path.abspath('./scripts')) #if in root
import preprocessing_time_series as pts


os.chdir(pts.working_dir() + 'Feature_releases/release_v02')

Populating the interactive namespace from numpy and matplotlib


Возьмем фичи и разложим на трансформации, чтобы выкинуть ненужные шаги

In [6]:
feat_imp = pd.concat([pd.read_csv('feat_imp_week10_xgboost_release_v02.csv', header=None).head(50), 
            pd.read_csv('feat_imp_week11_xgboost_release_v02.csv', header=None).head(50)])
feat_imp.head()

,0,1
0,Cliente_ID,8951
1,Ruta_SAK,5255
2,Producto_ID,2459
3,weight,1892
4,Log_Demanda_4last,1414


Теперь посмотрим, от каких этапов стоит избавиться

Соберем все вместе и посмотрим, какие трансформации нужны, а какие нет

In [47]:
all_features = pd.read_csv('train_MORELOS.csv', nrows=20).columns

def transformaton_types(feat_list):
    features = pd.DataFrame(feat_list, columns = ['Features'])
    features['Original'] = features['Features']
    features['LagWidth'] = features['Features'].str.extract('(\d)last', expand=True)
    features.loc[~features.LagWidth.isnull(), 'Original'] = features['Original'].str.replace('_(\d)last', '')
    features['Lag'] = features.loc[features.LagWidth.isnull(), 'Features'].str.extract('_(\d)', expand=True)
    features.loc[~features.Lag.isnull(), 'Original'] = features['Original'].str.replace('_(\d)', '')
    features['Split'] = features['Original'].apply(lambda f: f.split('_Mean_')[1] if '_Mean_' in f else np.nan)
    features['Original'] = features['Features'].apply(lambda f: f.split('_Mean_')[0])
    return features


features = transformaton_types(all_features)
features['Important'] = features.Features.isin(feat_imp[0].drop_duplicates().values).astype(int)
features

,Features,Original,LagWidth,Lag,Split,Important
0,Agencia_ID,Agencia_ID,NaN,NaN,NaN,1
1,Canal_ID,Canal_ID,NaN,NaN,NaN,1
2,Cliente_ID,Cliente_ID,NaN,NaN,NaN,1
3,Demanda_uni_equil,Demanda_uni_equil,NaN,NaN,NaN,0
4,Dev_proxima,Dev_proxima,NaN,NaN,NaN,0
5,Dev_proxima_by_uni,Dev_proxima_by_uni,NaN,NaN,NaN,0
6,Dev_uni_proxima,Dev_uni_proxima,NaN,NaN,NaN,0
7,Log_Demanda,Log_Demanda,NaN,NaN,NaN,0
8,Log_Demanda_Mean_Producto_ID_Cliente_ID_Agenci...,Log_Demanda,NaN,NaN,Producto_ID_Cliente_ID_Agencia_ID,0
9,Log_Demanda_Mean_Producto_ID_Cliente_ID_Ruta_SAK,Log_Demanda,NaN,NaN,Producto_ID_Cliente_ID_Ruta_SAK,0


In [48]:
pd.crosstab(features.Split, features.Important)

Important,0,1
Split,,
Producto_ID_Cliente_ID_Agencia_ID,14,10
Producto_ID_Cliente_ID_Ruta_SAK,15,9


In [49]:
originals = pd.crosstab([features.Split, features.Original], features.Important)
originals['Share'] = originals[1]/originals.sum(1)
originals = originals.sort_values(['Share', 0], ascending=[True, False])
originals

Important                                            0  1     Share
Split                             Original                         
Producto_ID_Cliente_ID_Agencia_ID Log_Venta_uni_hoy  6  0  0.000000
                                  Ordered            6  0  0.000000
Producto_ID_Cliente_ID_Ruta_SAK   Log_Venta_uni_hoy  6  0  0.000000
                                  Ordered            6  0  0.000000
                                  No_remains         2  4  0.666667
Producto_ID_Cliente_ID_Agencia_ID Log_Demanda        1  5  0.833333
                                  No_remains         1  5  0.833333
Producto_ID_Cliente_ID_Ruta_SAK   Log_Demanda        1  5  0.833333

В итоге сплиты я подобрал не так уж и плохо: я и так собирался выкинуть Log_Venta_uni_hoy и Ordered

In [36]:
pd.crosstab(features.Lag, features.Important)

Important,0,1
Lag,,
1,10,10
2,10,10
3,13,7


In [37]:
pd.crosstab(features.LagWidth, features.Important)

Important,0,1
LagWidth,,
3,9,11
4,9,11


In [43]:
originals = pd.crosstab(features.Original, features.Important)
originals['Share'] = originals[1]/originals.sum(1)
originals = originals.sort_values(['Share', 0], ascending=[True, False])
originals

Important,0,1,Share
Original,,,
Log_Venta_uni_hoy,18,0,0.000000
Ordered,18,0,0.000000
Dev_uni_proxima,1,0,0.000000
Semana,1,0,0.000000
State,1,0,0.000000
Town,1,0,0.000000
Log_Dev_proxima,5,1,0.166667
Log_Venta_hoy,5,1,0.166667
No_remains,9,9,0.500000


Избавлюсь от Log_Venta_uni_hoy, Ordered, Dev_uni_proxima и Town

Плюс ко всему после каждого этапа буду оставлять только нужные фичи: например, лаги считать только на переменных, по которым лаги нужны